In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, classification_report
import numpy as np
import os
os.chdir('Set home path here')

model_results={}
svr_model={}

#Confusion Matrix
def con_matrix(filename):
    #Crosstable between observed and predicted classes
    allocations=['Weak Inhibition','Moderate Inhibition','Strong Inhibition',
                    'Weak Induction','Moderate Induction','Strong Induction','No Interaction']
    allocations_order=['No Interaction','Weak Inhibition','Weak Induction','Moderate Inhibition','Moderate Induction',
                                   'Strong Inhibition','Strong Induction']
    report_order=['No Interaction','Weak Inhibition','Weak Induction','Moderate Inhibition','Moderate Induction',
                                   'Strong Inhibition','Strong Induction']
    output=pd.read_csv(filename)
    ct=pd.crosstab(output['Pred Category'],output['Obs Category'],dropna=False)
    ct=ct.reindex(allocations_order,axis="columns")
    ct=ct.reindex(allocations_order,axis="rows")
    #for allocation in allocations:
    #    ct[allocation]=(ct[allocation]/np.sum(ct[allocation]))*100 #Converting to %
    #Creating datafrane for calculating classification metrics
    report=classification_report(output['Obs Category'],output['Pred Category'],output_dict=True)
    report=pd.DataFrame.from_dict(report)
    report=report.transpose()
    report.drop(columns=['support'],inplace=True)
    report.drop('macro avg',inplace=True)
    report.drop('accuracy',inplace=True)
    report.drop('weighted avg',inplace=True)
    report.drop(columns=['f1-score'],inplace=True)
    print(report.columns)
    report=report.reindex(report_order,axis="rows")
    #Plotting crosstable heatmap and scoring bar chart
    fig, ax = plt.subplots()
    ax=sns.heatmap(ct,annot=True,cmap='Blues')
    ax.xaxis.set_tick_params(rotation=60)
    plt.show()

In [ ]:
import pickle
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split

input_data=pd.read_csv('Input Data.csv')
timeseries_features=pd.read_csv('Timeseries features.csv',index_col=['Unnamed: 0'])
humanpk=pd.read_excel('Substrates.xlsx',sheet_name='Human PK values')
oeselma=pd.read_csv('oeselma_output.csv')
in_vitro_data=pd.read_excel('Substrates.xlsx',sheet_name='Perpr Interactions')
#Merging Replacing rat pk values with human pk values (timeseries_features is the main featureset)
humanpk.drop(columns=humanpk.columns[0:6],inplace=True)
timeseries_features=pd.merge(timeseries_features,humanpk,left_index=True,right_index=True)
timeseries_features=timeseries_features[timeseries_features.columns.drop(list(timeseries_features.filter(regex='Rat')))]
print(timeseries_features.iloc[:,1:10])

#Creating different featuresets for comparison
#non-oeselma features
nonoeselma_features=timeseries_features.copy()
drop_perp=oeselma.columns
drop_sub=oeselma.columns
drop_perp = [f'{col} Perp' for col in drop_perp]
exceptions_perp=['Name Perp', 'Ion class Perp', 'Min. dist DD Perp', 'Min. dist DA Perp', 'Min. dist AA Perp']
exceptions_sub=['Name Sub', 'Ion class Sub', 'Min. dist DD Sub', 'Min. dist DA Sub', 'Min. dist AA Sub']
drop_perp = [i for i in drop_perp if i not in exceptions_perp]
drop_sub =[f'{col} Sub' for col in drop_sub]
drop_sub=[i for i in drop_sub if i not in exceptions_sub] 
#oeselma
oeselma_features=pd.merge(timeseries_features[drop_sub],timeseries_features[drop_perp],left_index=True,right_index=True)
#cyp and %fm features
cyp_data=timeseries_features.filter(regex=' T ')
fm_data=timeseries_features.filter(regex=r' %fm ')
cyp_features=pd.merge(cyp_data,fm_data,left_index=True,right_index=True)
#in vitro data
dfs=[]
for col in in_vitro_data.columns[2:10]:
    dfs.append(timeseries_features.filter(regex=f'{col}'))
in_vitro_features=pd.concat(dfs, axis=1)
#ECFP4 data
ecfp4=timeseries_features.filter(regex='ECFP4')
#Combined Chemical Feautures
chem=ecfp4.join(oeselma_features)


In [ ]:
#SVR model
def train_svr(features):   
    param_grid={'svr__C':[1e-2, 1e-1, 1.0, 10.0, 100.0,500],
                        'svr__epsilon':[0.01,0.05,0.1,0.5,1,10],
                        'svr__kernel':['linear','poly','rbf','sigmoid','precomputed']}
    svr_pipeline=make_pipeline(StandardScaler(),SVR())
    print('1')
    inner=KFold(n_splits=5,shuffle=True,random_state=0)
    search_results=GridSearchCV(svr_pipeline,param_grid,cv=inner,n_jobs=-1,scoring='neg_root_mean_squared_error').fit(features,input_data['log AUC Ratio'])
    print('2')
    optimal_params=search_results.best_params_
    optimal_params['C']=optimal_params.pop('svr__C')
    optimal_params['epsilon']=optimal_params.pop('svr__epsilon')
    optimal_params['kernel']=optimal_params.pop('svr__kernel')
    print('3')
    if features.equals(timeseries_features):
        svr_model['timeseries_features']=make_pipeline(StandardScaler(),SVR(**optimal_params))
    elif features.equals(cyp_features):
        svr_model['cyp_features']=make_pipeline(StandardScaler(),SVR(**optimal_params))
    elif features.equals(in_vitro_features):
        svr_model['in_vitro_features']=make_pipeline(StandardScaler(),SVR(**optimal_params))
    elif features.equals(chem):
        svr_model['chem']=make_pipeline(StandardScaler(),SVR(**optimal_params))
    print('4')
    return(svr_model) 
train_svr(chem)                            


In [ ]:
from sklearn.model_selection import train_test_split

def shap_importance(model_name,features):
    X_train,X_test,Y_train,Y_test = train_test_split(features,input_data['log AUC Ratio'], test_size=0.2, random_state=0)
    model_name.fit(X_train,Y_train)    
    X_train_summary = shap.kmeans(X_train, 5)
    explainer = shap.KernelExplainer(model_name.predict,X_train_summary)
    shap_values = explainer.shap_values(X_test)
    shap.summary_plot(shap_values, X_test, plot_type='bar')
    model_results[model_name]=shap_values
    shap.summary_plot(model_results[model_name], X_test)
    return(model_results)
shap_importance(svr_model['chem'],chem)